# Knight's Algorithm

Finding concordances, discordances, and ties in O(n log n) time using a modified merge sort algorithm. Knight's algorithm can be used to compute Kendall's tau, and many other similar rank correlation coefficients that are based on concordances. 

### Computing ties

Efficient way of computing ties. Need left ties, right ties, and joint ties. We can easily compute them in O(A \* B) time, but the beauty of the modified merge step is that it can compute the number of concordances, discordances in O(A \* B) time but with a lower constant factor.

In [1]:
def flatten(non_flat_list):
    """Flatten a list of lists into a simple list. Ex. the list
    [1, [2, [3, [4, 5]]], 6, [[7]]] is transformed into the simple
    list [1, 2, 3, 4, 5, 6, 7].
    """
    if not isinstance(non_flat_list, list):
        return [non_flat_list]
    results = []
    for item in non_flat_list:
        results.extend(flatten(item))
    return results


def efficient_ties(A, B):
    """Compute the number of ties between A and B in O(A + B) time.
    """
    
    # Create the initial count dict for B
    count_dict = {}
    for item in B:
        if item not in count_dict.keys():
            count_dict[item] = 0
        else:
            if isinstance(count_dict[item], list):
                count_dict[item].append(0)
            else:
                count_dict[item] = [0, 0] 
    
    # Fill in the count dict with A
    for item in A:
        if item in count_dict.keys():
            if isinstance(count_dict[item], list):
                for i in range(len(count_dict[item])):
                    count_dict[item][i] += 1
            else:
                count_dict[item] += 1      
                
    return sum(flatten(count_dict.values()))


# O(A) or O(B) is 4
# O(A * B) is 16
A = [3, 4, 7, 9]
B = [3, 3, 5, 8]
efficient_ties(A, B)

2

Now we can get left, right ties in O(A + B) time - problem: what about joint ties? We will have to deal with those in the merge step by modifying the algorithm that just counts inversions ...

## Merge step

The key to merge sort - since we recursively break the input array into two subarrays, we only run the merge step log n times. If the merge step itself takes O(n) time (or, if we consider n to be len(A) + len(B), O(A + B) time works too), we get a procedure that runs in O(n log n) time.

In this case, we've taken input arrays X and Y, translated them into ranks, and combined them into a list of tuples with the `zip` function, and sorted them on X (with Y as a secondary key). Ties associated with X are 'left' ties, ties associated with Y are 'right' ties, and joint ties are ties on both X and Y for a given pair.

In [4]:
def merge(B, C):
    """Merge sorted arrays B and C into a new sorted array D. O(len(B + C)),
    also report the number of inversions that have occured thus far.
    """
    
    # Calculate the number of left ties
    left_ties = efficient_ties([x[0] for x in B], [x[0] for x in C])
    
    # Calculate the number of concordances, discordances, right ties,
    # and joint ties
    D, discordances_at_column, rows_remaining = [], 0, len(B)
    discordances, right_ties, joint_ties = len(B) * len(C), 0, 0
    while len(B) > 0 and len(C) > 0:
        left_B, right_B = B[0]
        left_C, right_C = C[0]
        
        # Right / joint ties
        if right_B == right_C:
            tie = True
            while tie:
                
            
        if first_B <= first_C:
            D.append(B.pop(0))
            discordances += discordances_at_column
            rows_remaining -= 1
            if first_B == first_C:
                ties += 1
        else:
            D.append(C.pop(0))
            discordances_at_column += 1
    if B:
        D.extend(B)
        discordances += discordances_at_column * rows_remaining
    elif C:
        D.extend(C)
    concordances = concordances - discordances - ties
    return D, concordances, discordances, ties

In [1]:
# Y: Example A, B (gives 'right' ties)
B = [3, 4, 7, 9]
C = [3, 3, 5, 8]

# X: Example ranks (gives 'left' ties)
B_ranks = [1, 2.5, 2.5, 4]
C_ranks = [5.5, 5.5, 7, 8]

# Data structure for input is a list of tuples of the form [(x1, y1), (x2, y2), ...]
B = list(zip(B_ranks, B))
C = list(zip(C_ranks, C))

print B
print C

[(1, 3), (2.5, 4), (2.5, 7), (4, 9)]
[(5.5, 3), (5.5, 3), (7, 5), (8, 8)]


In [19]:
# Y: Example A, B (gives 'right' ties)
B = [3, 4, 7, 9]
C = [3, 3, 5, 8]

# X: Example ranks (gives 'left' ties)
B_ranks = [1, 2.5, 2.5, 4]
C_ranks = [5.5, 5.5, 7, 8]

# Data structure for input is a list of tuples of the form [(x1, y1), (x2, y2), ...]
B = list(zip(B_ranks, B))
C = list(zip(C_ranks, C))

In [24]:
[x[0] for x in B]

[1, 2.5, 2.5, 4]

In [25]:
[x[0] for x in C]

[5.5, 5.5, 7, 8]